In [1]:
import pickle
import random
import numpy as np
import networkx as nx
from sklearn.decomposition import PCA

import torch

from smartsampling import *
from evaluation import *

In [2]:
seed = 1
device = "cuda" if torch.cuda.is_available() else "cpu"

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if device=="cuda":
    torch.cuda.manual_seed(seed)

In [3]:
node_features = pickle.load(open('data/node_features.pkl', 'rb'))
text_features = np.array(list(node_features.values()))

pca = PCA(n_components=100)
pca.fit(text_features)
emb_features = pca.transform(text_features)

nnodes = text_features.shape[0]
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
p_space = [0.2, 0.5, 1, 2, 5]
q_space = [0.2, 0.5, 1, 2, 5]
c_space = [1, 2, 3, 4, 5]

nsamplers = 1000
lr = 0.005
weight_decay = 5e-4
dropout = 0.4
nepoch = 100
penalty = 3e-4

ratio = 0.1
nfold = 1

In [5]:
fold = 1
train_G, prediction_links = pickle.load(open('link_prediction/input_{}.pkl'.format(fold), 'rb'))

model = SmartSampling(text_features, emb_features, p_space, q_space, c_space, 
                      nnodes, nsamplers, lr, weight_decay, dropout, device, 
                      train_G)

In [6]:
embeddings = model.train(0, penalty)
lp_acc, lp_f1 = lp_evaluate(embeddings, prediction_links)
print(0, lp_acc, lp_f1)
    
for i in range(1, 100):
    embeddings = model.train(1, penalty)
    lp_acc, lp_f1 = lp_evaluate(embeddings, prediction_links)
    print(i, lp_acc, lp_f1)

0 0.7390291262135922 0.7825946295697186
Epoch: 1, Gain: 1.3147, Time: 4.0730s
1 0.6852427184466019 0.7276088052428163
Epoch: 1, Gain: 1.2700, Time: 3.8648s
2 0.6434951456310679 0.6845360824742268
Epoch: 1, Gain: 1.2163, Time: 3.8221s
3 0.6009708737864078 0.6450164104335809
Epoch: 1, Gain: 1.2214, Time: 3.8830s
4 0.5687378640776699 0.6185814872059078
Epoch: 1, Gain: 1.1808, Time: 3.8308s
5 0.545242718446602 0.6037225042301183
Epoch: 1, Gain: 1.1550, Time: 3.8268s
6 0.5293203883495146 0.5965379494007991
Epoch: 1, Gain: 1.1251, Time: 3.8815s
7 0.523883495145631 0.602979274611399
Epoch: 1, Gain: 1.0868, Time: 3.8318s
8 0.5192233009708738 0.6078555590750713
Epoch: 1, Gain: 1.0503, Time: 3.8222s
9 0.5182524271844661 0.6156467854376453
Epoch: 1, Gain: 0.9864, Time: 3.8971s
10 0.5093203883495145 0.6168309325246398
Epoch: 1, Gain: 0.9338, Time: 3.8745s
11 0.5071844660194175 0.621307072515667
Epoch: 1, Gain: 0.9233, Time: 3.9555s
12 0.507378640776699 0.6275143150785495
Epoch: 1, Gain: 0.9091, Ti